In [1]:

import requests_cache

# Install cache with a backend of your choice
requests_cache.install_cache('nyt_cache', backend='sqlite', expire_after=86400)  # Cache expires after 1 day


import os
import re
import sys
import json
import logging
import sqlite3
import shutil
import errno
from glob import glob
from typing import List, Union
from urllib.parse import urlparse
from lxml import etree

CACHE_DIR = './raw_json/recipes'
os.makedirs(CACHE_DIR, exist_ok=True)
URL_NYT = 'https://cooking.nytimes.com'
CATEGORY_TYPES = ['special_diets', 'cuisines', 'meal_types', 'dish_types']

class NYTRecipeScraper:
    def __init__(self, db_path='recipes.db', force=False):
        self.force = force
        self.db_path = db_path
        self._validate_cache_path()
        self._connect_db()
        self._create_tables()

    def _connect_db(self):
        self.conn = sqlite3.connect(self.db_path)
        self.conn.execute('PRAGMA foreign_keys = ON')

    def _create_tables(self):
        cursor = self.conn.cursor()
        # Create recipes table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS recipes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                slug TEXT UNIQUE,
                name TEXT,
                image_path TEXT,
                description TEXT,
                total_time_string TEXT,
                servings TEXT,
                rating_value REAL,
                rating_count INTEGER,
                ingredients TEXT,
                instructions TEXT,
                author TEXT
            )
        ''')
        # Create categories table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS categories (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT UNIQUE,
                type TEXT
            )
        ''')
        # Create recipe_categories table (many-to-many relationship)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS recipe_categories (
                recipe_id INTEGER,
                category_id INTEGER,
                FOREIGN KEY(recipe_id) REFERENCES recipes(id),
                FOREIGN KEY(category_id) REFERENCES categories(id),
                UNIQUE(recipe_id, category_id)
            )
        ''')
        self.conn.commit()

    def scrape_urls(self):
        recipe_urls = set()
        page = 1
        identical_page_failures = 0
        sequential_failures = 0

        while True:
            page_recipe_urls = set()
            url = f'{URL_NYT}/search?page={page}'
            try:
                props = self._fetch_page_props(url)
            except requests.exceptions.RequestException as e:
                logging.warning(e)
                print(f'Bad sequential response #{sequential_failures} for {url}')
                sequential_failures += 1
                if sequential_failures > 5:
                    print(f'Too many failures for {url}, continuing')
                    page += 1
                continue

            sequential_failures = 0
            results = props.get('results', [])
            recipes = [r for r in results if r.get('type') == 'recipe']

            print(f'Fetched {url} with {len(recipes)} recipes')
            if recipes:
                for recipe in recipes:
                    recipe_url = recipe.get('url')
                    if not recipe_url:
                        logging.warning(f'{url} is missing "url" in an article, skipping')
                        continue
                    if not re.search('^/recipes/', recipe_url):
                        continue
                    page_recipe_urls.add(recipe_url)
            else:
                print(f'No recipes found on URL {url}')

            if not page_recipe_urls:
                logging.warning(f'No recipe urls found for {url}')
                page += 1
                continue

            if page_recipe_urls.issubset(recipe_urls):
                identical_page_failures += 1
                print(f'Identical page response #{identical_page_failures} for {url}')
                if identical_page_failures >= 20:
                    print(f'Pages have been identical the last {identical_page_failures} times, stopping scrape on page {page}')
                    break
            else:
                identical_page_failures = 0
                recipe_urls.update(page_recipe_urls)

            page += 1

        # Save output as json file
        with open(os.path.join(CACHE_DIR, 'urls.json'), 'w') as f:
            json.dump({'urls': list(recipe_urls)}, f, indent=2)
        print(f'Completed {len(recipe_urls)} urls')

    def scrape_recipes(self):
        recipes_scraped = 0
        urls_file = os.path.join(CACHE_DIR, 'urls.json')

        if not os.path.exists(urls_file):
            print('"urls.json" does not exist. Run scrape_urls() first')
            sys.exit(1)

        print('Scraping recipes')

        with open(urls_file, 'r') as f:
            urls = json.load(f)['urls']

        for i, url in enumerate(urls):
            slug = os.path.basename(url)

            if not self.force and self._recipe_exists(slug=slug):
                continue

            cache_path = os.path.join(CACHE_DIR, slug)

            if not self.force and os.path.exists(cache_path):
                with open(cache_path, 'r') as f:
                    content = f.read()
            else:
                try:
                    content = self._fetch_url_content(f'{URL_NYT}{url}')
                except requests.exceptions.RequestException as e:
                    logging.exception(e)
                    logging.warning(f'ERROR scraping url {url}')
                    continue
                with open(cache_path, 'w') as fp:
                    fp.write(content)

            recipe_data = self._parse_page_props(content)
            if not recipe_data or 'recipe' not in recipe_data:
                logging.warning(f'ERROR parsing {url}')
                continue

            if i != 0 and i % 100 == 0:
                print(f'Scraped {i} recipes')

            recipe_file = os.path.join(CACHE_DIR, f'{slug}.json')
            with open(recipe_file, 'w', encoding='utf-8') as f:
                json.dump(recipe_data, f, ensure_ascii=False, indent=2)

            recipes_scraped += 1

        print(f'Scraped {recipes_scraped} recipes total')

    def scrape_images(self):
        recipe_files = self._get_recipe_json_files()
        for i, recipe_file in enumerate(recipe_files):
            with open(recipe_file, 'r', encoding='utf-8') as f:
                recipe_data = json.load(f)
            if not recipe_data or not recipe_data.get('recipe'):
                logging.warning(f'Skipping absent recipe record {recipe_file}')
                continue
            recipe = recipe_data['recipe']
            slug = os.path.basename(recipe['url'])

            if not self.force and self._recipe_exists(slug=slug):
                continue

            image_url = self._get_image_url_from_recipe(recipe)

            if not image_url or self._is_placeholder_image(image_url):
                logging.warning(f'Skipping absent image for {recipe_file}')
                continue

            try:
                response = requests.get(image_url, stream=True)
                response.raise_for_status()
            except Exception as e:
                logging.exception(e)
                print(f'Could not download image {image_url} for {recipe["name"]}')
                continue

            extension_match = re.match(r'.*(\.\w{3})$', os.path.basename(image_url))
            extension = extension_match.groups()[0] if extension_match else '.jpg'
            image_name = f'{slug}{extension}'
            image_path = os.path.join('images', image_name)
            os.makedirs('images', exist_ok=True)
            with open(image_path, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            if i != 0 and i % 100 == 0:
                print(f'Downloaded {i} images')

        print('Completed images')

    def ingest_recipes(self):
        num_ingested = 0

        recipe_files = self._get_recipe_json_files()
        for i, recipe_file in enumerate(recipe_files):
            if self._ingest_recipe(recipe_file):
                num_ingested += 1
            if i != 0 and i % 100 == 0:
                print(f'Ingested {num_ingested} recipes')

        print(f'Complete: ingested {num_ingested} recipes')

    def _ingest_recipe(self, recipe_file: str) -> bool:
        with open(recipe_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if 'recipe' not in data:
            logging.warning(f'Skipping absent recipe in file {recipe_file}')
            return False
        recipe = data['recipe']

        slug = os.path.basename(recipe['url'])

        # Set image path if it exists
        image_name = f'{slug}.jpg'
        image_path = os.path.join('images', image_name)
        image_url = image_path if os.path.exists(image_path) else None

        # Description, ingredients, & instructions
        description = self._replace_recipe_links_to_internal(recipe.get('topnote') or '')
        ingredients = self._replace_recipe_links_to_internal(self._get_ingredients_from_recipe(recipe))
        instructions = self._replace_recipe_links_to_internal(self._get_instructions_from_recipe(recipe))
        if not instructions or not ingredients:
            logging.warning(f'Skipping recipe {recipe_file} without ingredients or instructions')
            return False

        # Author
        author = recipe.get('contentAttribution', {}).get('cardByline', '')[:100]

        # Insert recipe into the database
        cursor = self.conn.cursor()
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO recipes (
                    slug, name, image_path, description, total_time_string,
                    servings, rating_value, rating_count, ingredients,
                    instructions, author
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                slug,
                recipe['title'],
                image_url,
                description,
                recipe.get('totalTime'),
                recipe['recipeYield'],
                recipe.get('ratings', {}).get('avgRating'),
                recipe.get('ratings', {}).get('numRatings'),
                json.dumps(ingredients, ensure_ascii=False),
                json.dumps(instructions, ensure_ascii=False),
                author
            ))
            self.conn.commit()
        except sqlite3.Error as e:
            logging.exception(e)
            return False

        # Get recipe_id
        cursor.execute('SELECT id FROM recipes WHERE slug = ?', (slug,))
        recipe_id = cursor.fetchone()[0]

        # Assign categories
        categories = [t['name'] for t in recipe.get('tags', []) if t.get('name')]
        for category_name in categories:
            category_name_lower = category_name.lower()
            # Insert category if it doesn't exist
            cursor.execute('''
                INSERT OR IGNORE INTO categories (name, type)
                VALUES (?, ?)
            ''', (category_name_lower, '_UNKNOWN_'))
            self.conn.commit()
            # Get category_id
            cursor.execute('SELECT id FROM categories WHERE name = ?', (category_name_lower,))
            category_id = cursor.fetchone()[0]
            # Insert into recipe_categories
            cursor.execute('''
                INSERT OR IGNORE INTO recipe_categories (recipe_id, category_id)
                VALUES (?, ?)
            ''', (recipe_id, category_id))
            self.conn.commit()

        return True

    def _fetch_page_props(self, url: str) -> dict:
        content = self._fetch_url_content(url)
        return self._parse_page_props(content)

    def _parse_page_props(self, content) -> dict:
        empty = {}
        html = etree.HTML(content)
        script_json = html.xpath('//script[@id="__NEXT_DATA__"]')
        if not script_json:
            return empty
        data = json.loads(script_json[0].text)
        return data.get('props', empty).get('pageProps', empty)

    def _fetch_url_content(self, url) -> str:
        response = requests.get(url, timeout=30)
        return response.content.decode()

    def _get_recipe_json_files(self) -> List[str]:
        return glob(os.path.join(CACHE_DIR, '*.json'))

    def _get_image_url_from_recipe(self, recipe: dict):
        if not recipe:
            return None
        if recipe.get('image') and recipe['image'].get('src'):
            return recipe['image']['src'].get('article')
        return None

    def _is_placeholder_image(self, image_path: str):
        return re.search('/assets/\\d+\\.(png|jpg|jpeg)', image_path, re.I)

    def _get_ingredients_from_recipe(self, recipe: dict) -> List:
        ingredients = []
        for ingredient in recipe.get('ingredients', []):
            if 'ingredients' in ingredient:
                if ingredient.get('name'):
                    ingredients.append(f'@@{ingredient["name"]}@@')
                for sub_ingredient in ingredient['ingredients']:
                    ingredients.append(f"{sub_ingredient['quantity']} {sub_ingredient['text']}")
            else:
                ingredients.append(f"{ingredient['quantity']} {ingredient['text']}")
        return ingredients

    def _get_instructions_from_recipe(self, recipe: dict) -> List:
        instructions = []
        for instruction in recipe.get('steps', []):
            if 'name' in instruction:
                if instruction.get('name'):
                    instructions.append(f'@@{instruction["name"]}@@')
                for inner_step in instruction.get('steps', []):
                    instructions.append(inner_step['description'])
            else:
                instructions.append(instruction['description'])
        return instructions

    def _recipe_exists(self, slug):
        cursor = self.conn.cursor()
        cursor.execute('SELECT 1 FROM recipes WHERE slug = ?', (slug,))
        return cursor.fetchone() is not None

    def _replace_recipe_links_to_internal(self, value: Union[str, list]) -> Union[str, list]:
        domain_parsed = urlparse(URL_NYT)
        re_search = rf'https?://{domain_parsed.hostname}/recipes/'
        re_replace = '/#/recipe/'
        if isinstance(value, list):
            return [re.sub(re_search, re_replace, x) for x in value]
        return re.sub(re_search, re_replace, value)

    def _validate_cache_path(self):
        try:
            os.makedirs(CACHE_DIR, exist_ok=True)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

    def close(self):
        self.conn.close()


In [ ]:
# Instantiate the scraper
scraper = NYTRecipeScraper(db_path='recipes.db', force=False)

# Step 1: Scrape recipe URLs
scraper.scrape_urls()

# Step 2: Scrape recipes based on the URLs collected
scraper.scrape_recipes()

# Step 3: Download recipe images
scraper.scrape_images()


In [ ]:
scraper.db_path = os.path.expanduser('recipes.db')
scraper._connect_db()
scraper._create_tables()
# Step 4: Ingest recipes into the SQLite database
scraper.ingest_recipes()


In [32]:

# Close the database connection when done
scraper.close()


In [ ]:
#copy all from /tmp/recipes to ./raw_json

In [38]:
#ADD INGREDIENTS TABLE AND REPARSE RAW JSON FILES EXTRACTING INGREDIENTS AND LINKING TO RECIPE_ID
conn = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ingredients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        recipe_id INTEGER,
        ingredient TEXT,
        FOREIGN KEY(recipe_id) REFERENCES recipes(id)
    )
''')
for recipe_file in glob('./raw_json/*.json'):
    with open(recipe_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if 'recipe' not in data:
        logging.warning(f'Skipping absent recipe in file {recipe_file}')
        continue
    recipe = data['recipe']
    slug = os.path.basename(recipe['url'])
    cursor.execute('SELECT id FROM recipes WHERE slug = ?', (slug,))
    recipe_id = cursor.fetchone()[0]
    ingredients = scraper._get_ingredients_from_recipe(recipe)
    for ingredient in ingredients:
        cursor.execute('''
            INSERT INTO ingredients (recipe_id, ingredient)
            VALUES (?, ?)
        ''', (recipe_id, ingredient))
conn.commit()
conn.close()

In [ ]:
import os
import sqlite3
import re
import json
import logging
from glob import glob

# Connect to the database
conn = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = conn.cursor()

# Create the unique_ingredients and recipe_unique_ingredients tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS unique_ingredients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ingredient TEXT UNIQUE
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS recipe_unique_ingredients (
        recipe_id INTEGER,
        unique_ingredient_id INTEGER,
        FOREIGN KEY(recipe_id) REFERENCES recipes(id),
        FOREIGN KEY(unique_ingredient_id) REFERENCES unique_ingredients(id),
        UNIQUE(recipe_id, unique_ingredient_id)
    )
''')

# Fetch all ingredients with their associated recipe IDs
cursor.execute('''
    SELECT recipe_id, ingredient FROM ingredients
''')
ingredient_rows = cursor.fetchall()

# Initialize a set to store unique ingredients and a list to map them
unique_ingredients_set = set()
ingredient_mapping = []

# Units list excluding units that are also ingredient names
unit_list = [
    'cup', 'cups', 'tablespoon', 'tablespoons', 'tbsp', 'tbsps',
    'teaspoon', 'teaspoons', 'tsp', 'tsps', 'pound', 'pounds', 'lb', 'lbs',
    'ounce', 'ounces', 'oz', 'gram', 'grams', 'g', 'kg', 'kilogram', 'kilograms',
    'ml', 'l', 'liter', 'liters', 'can', 'cans', 'package', 'packages',
    'quart', 'quarts', 'pint', 'pints', 'gal', 'gallon', 'gallons',
    'strip', 'strips', 'envelope', 'envelopes', 'jar', 'jars', 'pack', 'packs',
    'serving', 'servings', 'container', 'containers', 'box', 'boxes',
    'packet', 'packets', 'sheet', 'sheets', 'stalk', 'stalks', 'rib', 'ribs',
    'inch', 'inches'
]

# Words to ignore during parsing
ignore_words = {'x', 'small', 'medium', 'large', 'extra', 'extra-large', 'of', 'plus', 'and', '/'}

# Loop through each ingredient and process it
for recipe_id, ingredient_text in ingredient_rows:
    # Remove parenthetical notes
    ingredient_text = re.sub(r'\(.*?\)', '', ingredient_text)

    # Split the ingredient text into words
    words = ingredient_text.strip().split()
    index = 0

    # Remove leading quantities and units
    while index < len(words):
        word = words[index].lower()
        # Check if word is a quantity (number or fraction)
        if re.match(r'^\d+/\d+$', word) or re.match(r'^\d+(\.\d+)?$', word):
            index += 1
        # Check if word is a measurement unit or descriptor
        elif word in ignore_words:
            index += 1
        elif word.rstrip('s') in [unit.rstrip('s') for unit in unit_list]:
            index += 1
        else:
            break

    # Extract the remaining words as the ingredient
    ingredient_clean = ' '.join(words[index:])

    # Remove any trailing descriptors
    ingredient_clean = re.split(
        r',|\bto taste\b|\bfor serving\b|\bon\b|\bdivided\b|\bhalved\b|\bquartered\b',
        ingredient_clean,
        flags=re.I
    )[0]

    # Remove extra whitespace and convert to lowercase
    ingredient_clean = ingredient_clean.strip().lower()

    if ingredient_clean:
        unique_ingredients_set.add(ingredient_clean)
        ingredient_mapping.append((recipe_id, ingredient_clean))
    else:
        # If cleaning results in empty string, use the last word of the original ingredient
        ingredient_clean = words[-1].lower()
        unique_ingredients_set.add(ingredient_clean)
        ingredient_mapping.append((recipe_id, ingredient_clean))
        logging.warning(f'Ingredient "{ingredient_text}" cleaned to empty string, using "{ingredient_clean}" instead')

# Insert unique ingredients into the database and build a mapping
unique_ingredient_id_map = {}
for ingredient in unique_ingredients_set:
    cursor.execute('''
        INSERT OR IGNORE INTO unique_ingredients (ingredient)
        VALUES (?)
    ''', (ingredient,))
    # Get the id of the inserted ingredient
    cursor.execute('SELECT id FROM unique_ingredients WHERE ingredient = ?', (ingredient,))
    unique_id = cursor.fetchone()[0]
    unique_ingredient_id_map[ingredient] = unique_id

conn.commit()

# Insert the relationships into recipe_unique_ingredients
for recipe_id, ingredient in ingredient_mapping:
    unique_ingredient_id = unique_ingredient_id_map[ingredient]
    cursor.execute('''
        INSERT OR IGNORE INTO recipe_unique_ingredients (recipe_id, unique_ingredient_id)
        VALUES (?, ?)
    ''', (recipe_id, unique_ingredient_id))

conn.commit()
conn.close()

print(f'Found {len(unique_ingredients_set)} unique ingredients')


In [42]:
conn.close()

In [ ]:
# remove all @ signs from unique ingredients


In [ ]:
#print all unique ingredients that start with "cup"
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'cup%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])

In [ ]:
import os
import sqlite3
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Connect to the database
db_path = os.path.expanduser('~/recipes.db')
db = sqlite3.connect(db_path)
cursor = db.cursor()

# Fetch all unique ingredients starting with 'cup'
cursor.execute('''
    SELECT id, ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'cup%'
''')
rows = cursor.fetchall()

# Regular expression to remove 'cup' and measurements
def clean_ingredient(ingredient):
    original = ingredient  # Keep the original for logging
    # Remove 'cup' or 'cups' at the beginning
    ingredient = re.sub(r'^cups?\b', '', ingredient, flags=re.I).strip()
    # Remove any measurements (numbers, fractions, units) at the beginning
    ingredient = re.sub(r'^[/\d\s\w\.]+(grams?|g|kilograms?|kg|milliliters|ml|ounces?|oz|sticks?|tablespoons?|tbsp|teaspoons?|tsp)\b', '', ingredient, flags=re.I).strip()
    # Remove any remaining leading numbers or symbols
    ingredient = re.sub(r'^[\d/\s\.]+', '', ingredient)
    # Remove any leading slashes or punctuation
    ingredient = ingredient.lstrip('/').strip()
    # Convert to lowercase
    ingredient = ingredient.lower()
    if not ingredient:
        logging.warning(f'Ingredient "{original}" cleaned to empty string')
    return ingredient

# Mapping of old ingredient IDs to new ingredient names
updates = {}
for row in rows:
    ingredient_id = row[0]
    ingredient_name = row[1]
    cleaned_name = clean_ingredient(ingredient_name)
    if cleaned_name:
        updates[ingredient_id] = cleaned_name
    else:
        logging.warning(f'Ingredient "{ingredient_name}" cleaned to empty string and will be removed')
        updates[ingredient_id] = None  # Mark for deletion

# Now, update the unique_ingredients table
# But first, check for potential duplicates
# Build a mapping from cleaned_name to existing ingredient_id
cursor.execute('SELECT id, ingredient FROM unique_ingredients')
all_ingredients = cursor.fetchall()
ingredient_name_to_id = {}
for ingredient in all_ingredients:
    ingredient_name_to_id[ingredient[1]] = ingredient[0]

# Prepare updates
ingredient_id_changes = {}  # Maps old ingredient_id to new ingredient_id (in case of duplicates)
for ingredient_id, cleaned_name in updates.items():
    if cleaned_name is None:
        # Delete the ingredient as it cleaned to empty string
        cursor.execute('DELETE FROM unique_ingredients WHERE id = ?', (ingredient_id,))
    elif cleaned_name in ingredient_name_to_id:
        # The cleaned name already exists in the table
        existing_id = ingredient_name_to_id[cleaned_name]
        ingredient_id_changes[ingredient_id] = existing_id
        # Delete the old ingredient entry
        cursor.execute('DELETE FROM unique_ingredients WHERE id = ?', (ingredient_id,))
    else:
        # Update the ingredient name
        cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE id = ?', (cleaned_name, ingredient_id))
        # Update the mapping
        ingredient_name_to_id[cleaned_name] = ingredient_id

# Update the recipe_unique_ingredients table to reflect any ingredient_id changes
for old_id, new_id in ingredient_id_changes.items():
    # Update any entries in recipe_unique_ingredients where unique_ingredient_id is old_id
    cursor.execute('''
        UPDATE OR IGNORE recipe_unique_ingredients
        SET unique_ingredient_id = ?
        WHERE unique_ingredient_id = ?
    ''', (new_id, old_id))
    # Delete any duplicate entries that may have arisen
    cursor.execute('''
        DELETE FROM recipe_unique_ingredients
        WHERE rowid NOT IN (
            SELECT MIN(rowid)
            FROM recipe_unique_ingredients
            GROUP BY recipe_id, unique_ingredient_id
        )
    ''')

# Commit changes and close the connection
db.commit()
db.close()

print('Ingredient names starting with "cup" have been cleaned and the database updated.')


In [ ]:
#list all ingredients that start with "to"
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'to%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])
    
db.close()  

In [ ]:
import os
import sqlite3
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Connect to the database
db_path = os.path.expanduser('~/recipes.db')
db = sqlite3.connect(db_path)
cursor = db.cursor()

# Fetch all unique ingredients starting with 'to'
cursor.execute('''
    SELECT id, ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'to%'
''')
rows = cursor.fetchall()

# Regular expression to clean ingredients starting with 'to'
def clean_ingredient(ingredient):
    original = ingredient  # Keep the original for logging
    # Remove leading 'to' and any following numbers, measurements, or descriptors
    ingredient = re.sub(
        r'^to\b\s*(\d+(\.\d+)?\s*(x\s+)?(small|medium|large|extra-large|extra large)?\s*(\d+/\d+\s*)?(grams?|g|kilograms?|kg|milliliters|ml|ounces?|oz|sticks?|tablespoons?|tbsp[s]?|teaspoons?|tsp[s]?|pounds?|lb[s]?|cups?|cup[s]?)?)?\s*',
        '',
        ingredient,
        flags=re.I
    ).strip()

    # Remove any HTML tags or URLs
    ingredient = re.sub(r'<[^>]+>', '', ingredient)
    ingredient = re.sub(r'http[s]?://\S+', '', ingredient)

    # Remove any text after certain phrases
    ingredient = re.split(
        r',|\bto taste\b|\bfor serving\b|\bon\b|\bdivided\b|\bhalved\b|\bquartered\b|\band\b|\bor\b|\bplus\b',
        ingredient,
        flags=re.I
    )[0].strip()

    # Remove any leading or trailing punctuation
    ingredient = ingredient.strip(' ,.-')

    # Convert to lowercase
    ingredient = ingredient.lower()

    if not ingredient:
        logging.warning(f'Ingredient "{original}" cleaned to empty string')
    else:
        logging.info(f'Cleaned "{original}" to "{ingredient}"')
    return ingredient

# Mapping of old ingredient IDs to new ingredient names
updates = {}
for row in rows:
    ingredient_id = row[0]
    ingredient_name = row[1]
    cleaned_name = clean_ingredient(ingredient_name)
    if cleaned_name:
        updates[ingredient_id] = cleaned_name
    else:
        logging.warning(f'Ingredient "{ingredient_name}" cleaned to empty string and will be removed')
        updates[ingredient_id] = None  # Mark for deletion

# Fetch all existing ingredients to handle duplicates
cursor.execute('SELECT id, ingredient FROM unique_ingredients')
all_ingredients = cursor.fetchall()
ingredient_name_to_id = {}
for ingredient in all_ingredients:
    ingredient_name_to_id[ingredient[1]] = ingredient[0]

# Prepare updates
ingredient_id_changes = {}  # Maps old ingredient_id to new ingredient_id (in case of duplicates)
for ingredient_id, cleaned_name in updates.items():
    if cleaned_name is None:
        # Delete the ingredient as it cleaned to empty string
        cursor.execute('DELETE FROM unique_ingredients WHERE id = ?', (ingredient_id,))
    elif cleaned_name in ingredient_name_to_id:
        # The cleaned name already exists in the table
        existing_id = ingredient_name_to_id[cleaned_name]
        ingredient_id_changes[ingredient_id] = existing_id
        # Delete the old ingredient entry
        cursor.execute('DELETE FROM unique_ingredients WHERE id = ?', (ingredient_id,))
    else:
        # Update the ingredient name
        cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE id = ?', (cleaned_name, ingredient_id))
        # Update the mapping
        ingredient_name_to_id[cleaned_name] = ingredient_id

# Update the recipe_unique_ingredients table to reflect any ingredient_id changes
for old_id, new_id in ingredient_id_changes.items():
    # Update any entries in recipe_unique_ingredients where unique_ingredient_id is old_id
    cursor.execute('''
        UPDATE OR IGNORE recipe_unique_ingredients
        SET unique_ingredient_id = ?
        WHERE unique_ingredient_id = ?
    ''', (new_id, old_id))
    # Delete any duplicate entries that may have arisen
    cursor.execute('''
        DELETE FROM recipe_unique_ingredients
        WHERE rowid NOT IN (
            SELECT MIN(rowid)
            FROM recipe_unique_ingredients
            GROUP BY recipe_id, unique_ingredient_id
        )
    ''')

# Commit changes and close the connection
db.commit()
db.close()

print('Ingredient names starting with "to" have been cleaned and the database updated.')


In [57]:
#update unique ingredients that contain "@" to remove the "@" sign
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT id, ingredient FROM unique_ingredients
    WHERE ingredient LIKE '%@%'
''')
rows = cursor.fetchall()
for row in rows:
    ingredient_id = row[0]
    ingredient_name = row[1]
    cleaned_name = ingredient_name.replace('@', '')
    cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE id = ?', (cleaned_name, ingredient_id))
db.commit()
db.close()

In [ ]:

#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'for%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])
    #remove them
    cursor.execute('DELETE FROM unique_ingredients WHERE ingredient = ?', (row[0],))
db.commit()
db.close()

In [67]:

# Regular expression to remove measurements
def clean_ingredient(ingredient):
    original = ingredient  # Keep the original for logging
    # Remove leading 'ounces', optional '/', any numbers and units, and optional 'grams'
    pattern = r'^ounces\s*/?\s*(\d+(\.\d+)?\s*(grams?|g)?)?\s*'
    # Replace with empty string
    ingredient = re.sub(pattern, '', ingredient, flags=re.I).strip()
    # Remove any remaining leading numbers and units
    ingredient = re.sub(r'^\d+(\.\d+)?\s*(grams?|g)?\s*', '', ingredient, flags=re.I).strip()
    # Convert to lowercase
    ingredient = ingredient.lower()
    return ingredient

#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'ounces%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])
    #update item
    cleaned_name = clean_ingredient(row[0])
    #check if cleaned_name exists
    cursor.execute('SELECT id FROM unique_ingredients WHERE ingredient = ?', (cleaned_name,))
    existing_id = cursor.fetchone()
    if existing_id:
        #delete the old ingredient
        cursor.execute('DELETE FROM unique_ingredients WHERE ingredient = ?', (row[0],))
    else:
        #update the ingredient
        cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE ingredient = ?', (cleaned_name, row[0]))
db.commit()
db.close()
    

In [71]:
from bs4 import BeautifulSoup
#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE '%a href%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])
    soup = BeautifulSoup(row[0], 'html.parser')
    cleaned_name = soup.get_text()
    #check if cleaned_name exists
    cursor.execute('SELECT id FROM unique_ingredients WHERE ingredient = ?', (cleaned_name,))
    existing_id = cursor.fetchone()
    if existing_id:
        #delete the old ingredient
        cursor.execute('DELETE FROM unique_ingredients WHERE ingredient = ?', (row[0],))
    else:
        #update the ingredient
        cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE ingredient = ?', (cleaned_name, row[0]))
db.commit()
db.close()

In [82]:

# Regular expression to remove measurements
def clean_ingredient(ingredient):
    original = ingredient  # Keep the original for logging
    # Remove leading 'ounces', optional '/', any numbers and units, and optional 'grams'
    pattern = r'^teaspoons\s*/?\s*(\d+(\.\d+)?\s*(grams?|g|milliliters)?)?\s*'
    # Replace with empty string
    ingredient = re.sub(pattern, '', ingredient, flags=re.I).strip()
    # Remove any remaining leading numbers and units
    ingredient = re.sub(r'^\d+(\.\d+)?\s*(grams?|g)?\s*', '', ingredient, flags=re.I).strip()
    # Convert to lowercase
    ingredient = ingredient.lower()
    return ingredient
#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE 'teaspoons%'
''')
rows = cursor.fetchall()
for row in rows:
    cleaned_name = clean_ingredient(row[0])
    if cleaned_name.startswith('/'):
        cleaned_name = cleaned_name.replace('/2 grams ', '', 1)

    cursor.execute('SELECT id FROM unique_ingredients WHERE ingredient = ?', (cleaned_name,))
    existing_id = cursor.fetchone()
    if existing_id:
        #delete the old ingredient
        cursor.execute('DELETE FROM unique_ingredients WHERE ingredient = ?', (row[0],))
    else:
        #update the ingredient
        cursor.execute('UPDATE unique_ingredients SET ingredient = ? WHERE ingredient = ?', (cleaned_name, row[0]))

In [85]:
#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE '% or %'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])

In [84]:
import os
import sqlite3
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Connect to the database
db_path = os.path.expanduser('~/recipes.db')
db = sqlite3.connect(db_path)
cursor = db.cursor()

# Fetch all unique ingredients containing ' or '
cursor.execute('''
    SELECT id, ingredient FROM unique_ingredients
    WHERE ingredient LIKE '% or %'
''')
rows = cursor.fetchall()

# Function to clean and split the ingredient
def split_ingredient(ingredient):
    # Split on ' or ' (case-insensitive)
    options = re.split(r'\s+or\s+', ingredient, flags=re.I)
    # Clean each option
    options = [opt.strip().lower() for opt in options]
    return options

# Function to clean ingredient names (optional)
def clean_ingredient_name(ingredient):
    # Remove leading quantities and units
    ingredient = re.sub(
        r'^(\d+(\.\d+)?|\d+/\d+)\s*(x\s+)?(small|medium|large|extra[-\s]?large)?\s*(cups?|grams?|g|kilograms?|kg|milliliters?|ml|ounces?|oz|sticks?|tablespoons?|tbsp[s]?|teaspoons?|tsp[s]?|pounds?|lb[s]?)?\s*',
        '',
        ingredient,
        flags=re.I
    ).strip()
    # Remove any parenthetical notes
    ingredient = re.sub(r'\(.*?\)', '', ingredient)
    # Remove text after certain phrases
    ingredient = re.split(
        r',|\bto taste\b|\bfor serving\b|\bon\b|\bdivided\b|\bhalved\b|\bquartered\b',
        ingredient,
        flags=re.I
    )[0]
    # Remove extra whitespace and convert to lowercase
    ingredient = ingredient.strip().lower()
    return ingredient

# Mapping of old ingredient IDs to new ingredient IDs
ingredient_id_changes = {}  # {old_id: [new_id1, new_id2, ...]}
new_ingredients_set = set()
for row in rows:
    ingredient_id = row[0]
    ingredient_name = row[1]
    options = split_ingredient(ingredient_name)
    logging.info(f'Splitting "{ingredient_name}" into {options}')
    ingredient_id_changes[ingredient_id] = []
    for option in options:
        # Optional: Clean the ingredient name further
        cleaned_option = clean_ingredient_name(option)
        # Skip empty options
        if not cleaned_option:
            continue
        # Check if the option already exists in unique_ingredients
        cursor.execute('SELECT id FROM unique_ingredients WHERE ingredient = ?', (cleaned_option,))
        result = cursor.fetchone()
        if result:
            # Ingredient already exists
            new_id = result[0]
        else:
            # Insert new ingredient
            cursor.execute('INSERT INTO unique_ingredients (ingredient) VALUES (?)', (cleaned_option,))
            new_id = cursor.lastrowid
            new_ingredients_set.add(cleaned_option)
        ingredient_id_changes[ingredient_id].append(new_id)

# Now update the recipe_unique_ingredients table
for old_id, new_ids in ingredient_id_changes.items():
    # Find all recipes that use the old ingredient_id
    cursor.execute('SELECT recipe_id FROM recipe_unique_ingredients WHERE unique_ingredient_id = ?', (old_id,))
    recipe_ids = cursor.fetchall()
    for recipe_id_tuple in recipe_ids:
        recipe_id = recipe_id_tuple[0]
        for new_id in new_ids:
            # Insert new relationship
            try:
                cursor.execute('''
                    INSERT INTO recipe_unique_ingredients (recipe_id, unique_ingredient_id)
                    VALUES (?, ?)
                ''', (recipe_id, new_id))
            except sqlite3.IntegrityError:
                # Relationship already exists
                pass
    # Delete the old relationship
    cursor.execute('DELETE FROM recipe_unique_ingredients WHERE unique_ingredient_id = ?', (old_id,))
    # Delete the old ingredient
    cursor.execute('DELETE FROM unique_ingredients WHERE id = ?', (old_id,))

# Commit changes and close the connection
db.commit()
db.close()

print('Ingredients containing "or" have been split and the database updated.')


Ingredients containing "or" have been split and the database updated.


In [87]:
#find all items starting with "to" in unique ingredients
db = sqlite3.connect(os.path.expanduser('~/recipes.db'))
cursor = db.cursor()
cursor.execute('''
    SELECT ingredient FROM unique_ingredients
    WHERE ingredient LIKE '%pound%'
''')
rows = cursor.fetchall()
for row in rows:
    print(row[0])

pounds/910 grams ripe peaches
1½ pounds wild salmon
1/4-pound lobsters
pound/114 grams unsalted butter
4-1/2 pounds clams
store-bought loaf pound cake
a homemade loaf pound cake
pounds/1.8 kilograms whole blueberries
